In [1]:
import xpress as xp
#xp.controls.outputlog = 0 # turn off output log

import numpy as np
import pylab as pl

### Solve the following Sudoku
$$
\begin{aligned}
&\begin{array}{|ccc|ccc|ccc|}
\hline 7 & . & 8 & 2 & 5 & . & . & . &  . \\
. & 4 &  . & 8 & . & . & 5 & . & .\\
5 & . & . & . & . & . & 6 & . & . \\
\hline
8 & . & . & 6 & . & . & . & 1 & 4 \\
. & 1 & 9 & . & . & . & 2 & 6 & . \\
2 & 6 & . & . & . & 1 & . & . & 5 \\
\hline
. & . & 1 & . & . & . & . & . & . \\
. & . & . & . & . & 8 & . & 5 & . \\
. & . & . & . & 3 & 4 & 7 & . & 9 \\
\hline
\end{array}
\end{aligned}
$$

In [2]:
size = 9
size_squared = size*size
square_size = 3

A = np.array([[7,0,8,2,5,0,0,0,0],
              [0,4,0,8,0,0,5,0,0],
              [5,0,0,0,0,0,6,0,0],
              [8,0,0,6,0,0,0,1,4],
              [0,1,9,0,0,0,2,6,0],
              [2,6,0,0,0,1,0,0,5],
              [0,0,1,0,0,0,0,0,0],
              [0,0,0,0,0,8,0,5,0],
              [0,0,0,0,3,4,7,0,9]])

## First Solution
Efficent but hard to read

In [54]:
sudoku = xp.problem()

In [55]:
x = np.array([[[xp.var(vartype=xp.binary) for _ in range(size)] for _ in range(size)] for _ in range(size)], dtype=xp.npvar)
sudoku.addVariable(x)

In [56]:
_ = [[sudoku.addConstraint(xp.Sum(x[i, j]) == 1) for i in range(size)] for j in range(size)]

In [57]:
_ = [[sudoku.addConstraint(xp.Sum(x[i, :, k]) == 1) for i in range(size)] for k in range(size)]

In [58]:
_ = [[sudoku.addConstraint(xp.Sum(x[:, j, k]) == 1) for j in range(size)] for k in range(size)]

In [59]:
_ = [[[sudoku.addConstraint(xp.Sum(x[i:i+square_size, j:j+square_size, k]) == 1) for i in range(0, size, square_size)] for j in range(0, size, square_size)] for k in range(size)]

In [60]:
for i in range(size):
    for j in range(size):
        if A[i, j] != 0:
            sudoku.addConstraint(x[i, j, A[i, j]-1] == 1)

In [61]:
sudoku.solve()

FICO Xpress v8.13.5, Community, solve started 5:07:19, Apr 29, 2022
Heap usage: 662KB (peak 662KB, 495KB system)
Minimizing MILP noname using up to 4 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
       352 rows          729 cols         2944 elements       729 globals
Presolved problem has:
        31 rows           39 cols          141 elements        39 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 698KB (peak 1293KB, 495KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [      0.0,       0.0] / [      0.0,       0.0]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 9.1GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual     

In [65]:
sol = np.array(sudoku.getSolution()).reshape(size, size, size).astype(int)
sol@np.arange(1, 10)

array([[7, 3, 8, 2, 5, 6, 4, 9, 1],
       [1, 4, 6, 8, 7, 9, 5, 3, 2],
       [5, 9, 2, 4, 1, 3, 6, 8, 7],
       [8, 5, 7, 6, 9, 2, 3, 1, 4],
       [3, 1, 9, 7, 4, 5, 2, 6, 8],
       [2, 6, 4, 3, 8, 1, 9, 7, 5],
       [9, 2, 1, 5, 6, 7, 8, 4, 3],
       [4, 7, 3, 9, 2, 8, 1, 5, 6],
       [6, 8, 5, 1, 3, 4, 7, 2, 9]])

## Second Solution
Less efficent but easier to read

In [ ]:
sudoku = xp.problem()

In [26]:
x = np.array([[xp.var(vartype=xp.integer, lb=1, ub=size) for _ in range(size)] for _ in range(size)], dtype=xp.npvar)
sudoku.addVariable(x)

In [27]:
def add_inequality_constraint(problem, x1, x2, M=size, eps=1):
    y = xp.var(vartype=xp.binary)
    problem.addVariable(y)
    problem.addConstraint(x1 - x2 >= eps - M * y)
    problem.addConstraint(x1 - x2 <= -eps + M * (1 - y))

def add_inequality_constraints_between_variables(problem, variables):
    vars = variables.ravel()
    for i in range(len(vars)):
        for j in range(i+1, len(vars)):
            add_inequality_constraint(problem, vars[i], vars[j])

In [28]:
# numbers on rows and columns must be different
for i in range(size):
    add_inequality_constraints_between_variables(sudoku, x[i])
    add_inequality_constraints_between_variables(sudoku, x[:, i])

# numbers in a square must be different
for i in range(0, size, square_size):
    for j in range(0, size, square_size):
        add_inequality_constraints_between_variables(sudoku, x[i:i+square_size, j:j+square_size])

In [29]:
for i in range(size):
    for j in range(size):
        if A[i, j] != 0:
            sudoku.addConstraint(x[i, j] == A[i, j])

In [30]:
sudoku.solve()

FICO Xpress v8.13.5, Community, solve started 4:59:51, Apr 29, 2022
Heap usage: 3573KB (peak 3573KB, 957KB system)
Minimizing MILP noname using up to 4 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
      1972 rows         1053 cols         5860 elements      1053 globals
Presolved problem has:
       657 rows          357 cols         1807 elements       357 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 3876KB (peak 4636KB, 957KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  9.00e+00] / [ 1.25e-01,  1.75e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  9.00e+00] / [ 1.25e-01,  9.00e+00]
  Objective      [min,max] : [      0.0,       0.0] / [      0.0,       0.0]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 9.1GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual  

In [31]:
sol = np.array(sudoku.getSolution()[:size_squared]).reshape(size, size).astype(int)

sol

array([[7, 3, 8, 2, 5, 6, 4, 9, 1],
       [1, 4, 6, 8, 7, 9, 5, 3, 2],
       [5, 9, 2, 4, 1, 3, 6, 8, 7],
       [8, 5, 7, 6, 9, 2, 3, 1, 4],
       [3, 1, 9, 7, 4, 5, 2, 6, 8],
       [2, 6, 4, 3, 8, 1, 9, 7, 5],
       [9, 2, 1, 5, 6, 7, 8, 4, 3],
       [4, 7, 3, 9, 2, 8, 1, 5, 6],
       [6, 8, 5, 1, 3, 4, 7, 2, 9]])